### Задание

Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует
2.(дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

In [18]:
!pip install rouge

In [19]:
import pandas as pd
import razdel

from transformers import AutoTokenizer, MBartForConditionalGeneration
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

In [2]:
from datasets import load_dataset

dataset = load_dataset('IlyaGusev/gazeta', revision="v1.0")["train"]

No config specified, defaulting to: gazeta/default
Found cached dataset gazeta (C:/Users/Uliana/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
df = pd.DataFrame(dataset)

In [5]:
data = df[['text', 'summary']]

In [6]:
data = data.sample(100)

In [8]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.47G [00:00<?, ?B/s]

In [9]:
print(type(tokenizer), type(model))

<class 'transformers.models.mbart.tokenization_mbart_fast.MBartTokenizerFast'> <class 'transformers.models.mbart.modeling_mbart.MBartForConditionalGeneration'>


In [10]:
article_text = data['text'].iloc[1]
article_text

'Московский арбитражный суд в четверг удовлетворил иск Росимущества по истребованию 16,387 га земли на западе Москвы у «Территориальной дирекции Сетуньская», акционером которой является Елена Батурина. Жена бывшего мэра Москвы намерена оспорить решение суда. «Компания владеет землями на абсолютно законном основании. В ближайшее время мы подадим апелляцию на это судебное решение», — заявил представитель предпринимательницы Геннадий Теребков. По его мнению, Росимущество так и не предоставило в суде доказательств реального существования истребуемых им земельных участков. «Вынесенное судом решение удивляет, но, тем не менее, до получения мотивировочной части давать конкретные комментарии по этому поводу считаем преждевременным», — сказал он. В Росимуществе не комментируют дальнейшие действия ведомства и планы по распоряжению земельными участками. Речь идет о трех участках (около 37 га) на пересечении Староволынской и Минской улиц, которые были переданы ТД «Сетуньская» в 2003 году. Тогда Мо

In [11]:
input_ids = tokenizer.prepare_seq2seq_batch(
    [article_text],
    src_lang="ru_XX", 
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=600
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=180,
    no_repeat_ngram_size=3,
    num_beams=5,
    top_k=0
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

C:\Users\Uliana\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3668: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [12]:
print('original: \n', data['summary'].iloc[1], '\n')
print('predict by model: \n', summary)

original: 
 Арбитражный суд Москвы удовлетворил иск Росимущества об изъятии 16 га столичной земли, принадлежащей супруге экс-мэра Москвы Елене Батуриной. Речь идет об участках, на которых планировалось построить дипломатические миссии 12 стран. Жена бывшего мэра намерена оспорить решение суда. Эксперты считают, что ее шансы минимальны. 

predict model: 
 Московский арбитражный суд удовлетворил иск Росимущества по истребованию 16,387 га земли на западе Москвы у «Территориальной дирекции Сетуньская», акционером которой является Елена Батурина. Жена бывшего мэра Москвы намерена оспорить решение суда.


Модель прересказала достаточно хорошо, смысл уложен в более компактный текст

In [14]:
def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [15]:
record = data.iloc[1]

In [20]:
def calc_lead_n_score(records, n=3, lower=True, nrows=10):
    references = []
    predictions = []

    for i, record in enumerate(records):
        if i >= nrows:
            break

        input_ids = tokenizer.prepare_seq2seq_batch(
            [data['text'].iloc[47]],
            src_lang="ru_XX", 
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=600
        )["input_ids"]

        output_ids = model.generate(
            input_ids=input_ids,
            max_length=162,
            no_repeat_ngram_size=3,
            num_beams=5,
            top_k=0
        )[0]

        summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

        references.append(summary)

        text = data['summary'].iloc[47]
        text = text if not lower else text.lower()
        sentences = [sentence.text for sentence in razdel.sentenize(text)]
        prediction = " ".join(sentences[:n])
        predictions.append(prediction)

    calc_scores(references, predictions)

calc_lead_n_score(data, n=1)

Count: 2
Ref: Российского посла в Судане Миргаяса Ширинского обнаружили с признаками острого сердечного приступа. Соболезнования в связи со смертью посла выразил директор Службы внешней разведки Сергей Нарышкин. Он отметил, что его хорошо знали как опытного дипломата, прекрасного специалиста, надежного товарища. Тем не менее правоохранительные органы Судана не исключали версию об отравлении российского посла.
Hyp: причиной смерти посла россии в судане миргаяса ширинского стал сердечный приступ, сообщили в мид россии. накануне, 23 августа, ширинского обнаружили мертвым в своей резиденции в хартуме. версия об отравлении российского дипломата не подтвердилась.
BLEU:  0.3318820893876461
ROUGE:  {'rouge-1': {'r': 0.14893617021276595, 'p': 0.25, 'f': 0.18666666198755566}, 'rouge-2': {'r': 0.04, 'p': 0.0625, 'f': 0.04878048304580653}, 'rouge-l': {'r': 0.1276595744680851, 'p': 0.21428571428571427, 'f': 0.15999999532088902}}


Вывод: Смысл модель уловила хорошо и даже попробовала предсказать будущее (версия об отравлении российского дипломата не подтвердилась.)